In [1]:
import torch
import torchvision
import math
import pandas as pd
import numpy as np
import gc

from tqdm import tqdm_notebook
from torch.utils.tensorboard import SummaryWriter


from GWD_data import WheatDataset
from metric import calculate_image_precision
from utils import collate_fn, plot_boxes, get_model_name, format_prediction_string, remove_blanks, is_contain_blanks
from config import config


In [2]:
conf = config()
torch.random.manual_seed(5)
writer = SummaryWriter("runs/Ex_Training_1") #


WD_Train = WheatDataset(conf)
WD_Valid = WheatDataset(conf, train=False)
WD_Train_Loader = torch.utils.data.DataLoader(WD_Train, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)
WD_Valid_Loader = torch.utils.data.DataLoader(WD_Valid, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)

In [3]:
def get_model(num_classes=2):
    print('New Model ')
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=True, pretrained_backbone=True)
    model.roi_heads.box_predictor.cls_score = torch.nn.Linear(model.roi_heads.box_predictor.cls_score.in_features, num_classes, bias=True)
    model.roi_heads.box_predictor.bbox_pred = torch.nn.Linear(model.roi_heads.box_predictor.bbox_pred.in_features, num_classes*4, bias=True)
    return model

In [5]:
#model_path=None
#GWD_Model = torch.load(model_path) if model_path else get_model()
#GWD_Model.load_state_dict(torch.load('model/model_rc/fasterrcnn_resnet50_fpn_best.pth'))
#GWD_Model.to(conf.DEVICE)
#EPOCH = int(model_path.split('/')[-1].split('_')[2])+1 if model_path else 0

#params = [p for p in GWD_Model.parameters() if p.requires_grad]
#optimizer = torch.optim.SGD(params, lr=0.0002,momentum=0.9, weight_decay=0.0005)

New Model 


In [4]:
model_path = 'model/model_rc/GWD_EPOCH_19_SCORE_0.0000_LOSS_201.4089.pt'
GWD_Model = torch.load(model_path) if model_path else get_model()
GWD_Model.to(conf.DEVICE)
EPOCH = int(model_path.split('/')[-1].split('_')[2])+1 if model_path else 0

params = [p for p in GWD_Model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0005,momentum=0.9, weight_decay=0.0005)

In [5]:
_iter=len(WD_Train_Loader)*(EPOCH)

for i in tqdm_notebook(range(EPOCH, 30, 1)):
    
    _epoch_loss=0
    _ = GWD_Model.train()
    for images, targets in tqdm_notebook(WD_Train_Loader):
        
        #if not is_contain_blanks(targets):
        try:
            images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
            targets = [{k: torch.tensor(v).to(conf.DEVICE) for k, v in target.items()} for target in targets]

            loss_dictionary = GWD_Model(images,targets)
            summed_loss = sum(loss for loss in loss_dictionary.values())
            summed_loss_value = summed_loss.item()


            if math.isfinite(summed_loss_value):
                optimizer.zero_grad()
                summed_loss.backward()
                optimizer.step()
                _epoch_loss+=summed_loss_value

            else:
                print('Loss is undefined:',summed_loss_value,'   skipping BackProp for step no:',_iter)
                print(loss_dictionary)


            writer.add_scalar('Running Loss/Summed', summed_loss_value, _iter)
            writer.add_scalar('Running Loss/Classifier', loss_dictionary['loss_classifier'].item(), _iter)
            writer.add_scalar('Running Loss/Box_Regress', loss_dictionary['loss_box_reg'].item(), _iter)
            writer.add_scalar('Running Loss/Objectness', loss_dictionary['loss_objectness'].item(), _iter)
            writer.add_scalar('Running Loss/RPN_Box_regress', loss_dictionary['loss_rpn_box_reg'].item(), _iter)
            
        except:
            print('Some weird shape error due to blank images, but handeled')
        
        _iter+=1
        
    torch.save(GWD_Model, 'model/model_rc/'+get_model_name(i, 0, _epoch_loss))
    torch.cuda.empty_cache()
    gc.collect()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



...

Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled


